# 01-2: External tools


In [6]:
PROJECT_ID = YOUR_PROJECT_ID  # <--- CHANGE THIS
LOCATION = "us-central1"  # <--- CHANGE THIS
MODEL_NAME = "text-bison@002" # <--- OPTIONALLY CHANGE HIS. OUTPUTS MAY DIFFER SIGNIFICANTLY

In [7]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project=PROJECT_ID,
              location=LOCATION)
parameters = {
    "temperature": 0,
    "max_output_tokens": 1024,
    "top_p": 0.8,
    "top_k": 40
}

model = TextGenerationModel.from_pretrained(MODEL_NAME)

In [8]:
def call_llm(model, parameters, llm_call, show_activity = True):
  response = model.predict(llm_call, **parameters).text

  if show_activity:
    BOLD = "\033[1m"
    UNFORMAT = "\033[0m\x1B[0m"
    print(f"{BOLD}The call to the LLM:{UNFORMAT}\n{llm_call}\n")
    print(f"{BOLD}The response:{UNFORMAT}")
    print(response)

  return response  # Return to `_` if not needed.

In [13]:
question = "Who is President of Argentina?" # Quien es el presidente de Argentina no va la wikitool
_ = call_llm(model, parameters, question)

The call to the LLM:
Who is President of Argentina?

The response:
 Alberto Fernández


### The Wikipedia Tool

The function below takes a query, returns the top Wikipedia article match for the query, and then retrieves the first `return_chars` characters of the article.

This tool is for teaching purposes and is somewhat limited. It cannot access lists or sidebars, does not handle suggestions well, does not support search within a Wikipedia article, and may not always return a result.

In [10]:
import wikipedia
def wiki_tool(query, return_chars = 1000):
  try:
    page = wikipedia.page(query, auto_suggest=False, redirect=True).content
  # If no exact match, take Wikipedia's auto-suggestion.
  except wikipedia.exceptions.PageError as e:
    page = wikipedia.page(query, auto_suggest=True, redirect=True).content
  snippet = page[0:return_chars]
  return snippet

Try the tool:

In [15]:
wiki_tool("president  of argentina")

'The president of Argentina (Spanish: Presidente de Argentina; officially known as the president of the Argentine Nation and president of the Argentine Republic, Spanish: Presidente de la Nación Argentina) is both head of state and head of government of Argentina. Under the national constitution, the president is also the chief executive of the federal government and commander-in-chief of the armed forces.\nThroughout Argentine history, the office of head of state has undergone many changes, both in its title as in its features and powers. The current president Javier Milei was sworn into office on 10 December 2023. He succeeded Alberto Fernández. \nThe constitution of Argentina, along with several constitutional amendments, establishes the requirements, powers, and responsibilities of the president, the term of office and the method of election.\n\n\n== History ==\nThe origins of Argentina as a nation can be traced to 1776, when it was separated by the King Charles III of Spain from t

### Chaining LLM Calls for Tool Use

A basic two-step tool use LLM chain contains a few pieces, broken down here step-by-step.

If you call the model (as of October 2023) with this example question about an obscure musician it hallucinates an incorrect answer:

In [16]:
question = "Quien publicó el album Struggle for Pleasure ?"
_ = call_llm(model, parameters, question)

The call to the LLM:
Quien publicó el album Struggle for Pleasure ?

The response:
 El álbum Struggle for Pleasure fue publicado por la banda de rock estadounidense The Replacements en 1989.


### Step 1: Provide the LLM Instructions for Using the Tool

You must provide the LLM both instructions for your task and for how to use the tool.

This "instructions" part of the LLM call is sometimes called the "context" or some variation of "condition" ("conditioning", "conditioning prompt").

In [ ]:
context = """Answer questions using a lookup of Wikipedia.
After each question, write a Wikipedia search followed by '<STOP>'.
The Wikipedia search will be used to retrieve the most relevant content.
A section of the Wikipedia article will then be sent to the next LLM call.
Use the text of the Wikipedia article to answer the question."""

### Step 2: Provide An Exemplar

The LLM needs exemplars that show how to use the tool to complete the task.

This example has only a one-shot exemplar, few-shot would be better.

The Wikipedia article text in this exemplar comes from running `wiki_tool("chancellor of germany")` in August 2023.

Note: After future retrainings the LLM will answer this question correctly without an external tool. But this one-shot exemplar will still work, since it shows the pattern of a Wikipedia search, a response, and an answer based on the response.

In [ ]:
exemplar = """Question: Who is Chancellor of Germany?
Wikipedia Search: chancellor of Germany<STOP>
Wikipedia Article: The chancellor of Germany, officially the federal chancellor of the Federal Republic of Germany, is the head of the federal government of Germany, and the commander in chief of the German Armed Forces during wartime. The chancellor is the chief executive of the Federal Cabinet and heads the executive branch. The chancellor is elected by the Bundestag on the proposal of the federal president and without debate (Article 63 of the German Constitution).The current officeholder is Olaf Scholz of the SPD, who was elected in December 2021, succeeding Angela Merkel. He was elected after the SPD entered into a coalition agreement with Alliance 90/The Greens and the FDP.\n\n\n== History of the office ==\nThe office of Chancellor has a long history, stemming back to the Holy Roman Empire, when the office of German archchancellor was usually held by archbishops of Mainz. The title was, at times, used in several states of German-speaking Europe. The modern office of chancellor was established with the
Answer: Olaf Scholz"""

### Step 3: Make the First Call in the LLM Chain

We'll combine our context and our exemplar together with our question and make a call to the LLM asking for a Wikipedia search query as a response.

In [ ]:
step_one_call = f"""{context}

{exemplar}

Question: {question}
Wikipedia Search:"""
step_one_response = call_llm(model, parameters, step_one_call)

### Step 4: Use the LLM's Response to Query the Tool

Note the LLM response contains more than the Wikipedia search query.

LLMs work by repeatedly predicting the next token over and over again, based on the tokens in the LLM call plus any previously predicted tokens. This means the LLM will generate excess text, it does not know to stop after the Wikipedia search query.

Everything beyond the Wikipedia search query is garbage. The excess text is discarded using the `<STOP>` signifier, though this could also be done with line breaks.

In a production system, it's important to control costs by limiting the response size when making an LLM call like this.

The following function takes the LLM response from the first chain step and returns the Wikipedia query.

In [ ]:
def get_wiki_query (llm_response, stop_text = "<STOP>"):
  # Assumes the query is in the first line.
  first_line = llm_response.splitlines()[0]
  query = first_line.split(stop_text)[0]
  return query.strip() # Remove leading and trailing whitespace.

Use this function on the response from the previous LLM call to extract the query, then  use `wiki_tool` to search Wikipedia.

In [ ]:
wiki_query = get_wiki_query(step_one_response)
print(f"Tool Query: {wiki_query}")
wiki_text = wiki_tool(wiki_query)
print(f"Wikipedia Snippet: {wiki_text}")

### Step 5: Use the Tool Response to Make the Second Call in the LLM Chain

Next, answer the question by taking the output from the tool and constructing a second LLM call.

LLM tool usage generally maintains the history of the previous calls and responses. To construct the second call in the chain:
1. Start with the first LLM call in the chain.
1. Append the previously generated Wikipedia query.
1. Append the Wikipedia search result.

Here's a reminder of what our first call looked like:

In [ ]:
print(step_one_call)

This first LLM call is combined with the query from the first LLM response and the output from the Wikipedia tool, along with structure to match the exemplar:

In [ ]:
step_two_call = f"""{step_one_call} {wiki_query}
Wikipedia Article: {wiki_text}
Answer: """
step_two_response = call_llm(model, parameters, step_two_call)

## Putting All the Steps Together

This code snippet below gathers all the steps above, dependent packages, and dependent functions into a single function that manages the two-step tool usage LLM chain.

You can copy and paste this code into your own project and it should work, assuming you've installed the right packages and authenticated.

In [ ]:
import wikipedia

def call_llm(model, parameters, llm_call, show_activity = True):
  # Wraps an LLM call to Vertex, optionally displaying the call and response.
  response = model.predict(llm_call, **parameters).text

  if show_activity:
    BOLD = "\033[1m"
    UNFORMAT = "\033[0m\x1B[0m"
    print(f"{BOLD}The call to the LLM:{UNFORMAT}\n{llm_call}\n")
    print(f"{BOLD}The response:{UNFORMAT}")
    print(response)

  return response  # Return to `_` if not needed.


def wiki_tool(query, return_chars = 1000):
  try:
    page = wikipedia.page(query, auto_suggest=False, redirect=True).content
  # If no exact match, take Wikipedia's suggestion.
  except wikipedia.exceptions.PageError as e:
    page = wikipedia.page(query, auto_suggest=True, redirect=True).content
  snippet = page[0:return_chars]
  return snippet


def get_wiki_query (llm_response, stop_text = "<STOP>"):
  # Extract the wikipedia query from the LLM response.
  # Assumes the query is in the first line.
  first_line = llm_response.splitlines()[0]
  query = first_line.split(stop_text)[0]
  return query.strip() # Remove leading and trailing whitespace


def wiki_tool_chain(model,
                    parameters,
                    context,
                    exemplar,
                    question,
                    show_activity=False):
  # Answer a query using wikipedia by calling an LLM.
  step_one_call = (
      f"{context}\n\n{exemplar}\n\nQuestion: {question}\nWikipedia Search:"
  )
  if show_activity:
    print("\033[1mMaking the first LLM call...\033[0m\x1B[0m")
  step_one_response = call_llm(model, parameters, step_one_call, show_activity)
  wiki_query = get_wiki_query(step_one_response)
  wiki_text = wiki_tool(wiki_query)

  step_two_call = (
      f"{step_one_call} {wiki_query}\nWikipedia Article: {wiki_text}\nAnswer: "
  )
  if show_activity:
    print("\033[1mMaking the second LLM call...\033[0m\x1B[0m")
  step_two_response = call_llm(model, parameters, step_two_call, show_activity)

  return step_two_response

An example using the code above:


In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel

# Outside this notebook, set PROJECT_ID, LOCATION, and MODEL_NAME.
# When running in the notebook, these are set in part 0.
vertexai.init(project=PROJECT_ID, location=LOCATION)
# These settings control how deterministic the LLM response is.
parameters = {
    "temperature": 0,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained(MODEL_NAME)

context = """Answer questions using a lookup of wikipedia.
After each question, write a wikipedia search followed by '<STOP>'.
The wikipedia search will be used to retrieve the most relevant content.
A section of the wikipedia article will then be sent to the next LLM call.
Use the text of the wikipedia article to answer the question."""

exemplar = """Question: Who is Chancellor of Germany?
Wikipedia Search: chancellor of Germany<STOP>
Wikipedia Article: The chancellor of Germany, officially the federal chancellor of the Federal Republic of Germany, is the head of the federal government of Germany, and the commander in chief of the German Armed Forces during wartime. The chancellor is the chief executive of the Federal Cabinet and heads the executive branch. The chancellor is elected by the Bundestag on the proposal of the federal president and without debate (Article 63 of the German Constitution).The current officeholder is Olaf Scholz of the SPD, who was elected in December 2021, succeeding Angela Merkel. He was elected after the SPD entered into a coalition agreement with Alliance 90/The Greens and the FDP.\n\n\n== History of the office ==\nThe office of Chancellor has a long history, stemming back to the Holy Roman Empire, when the office of German archchancellor was usually held by archbishops of Mainz. The title was, at times, used in several states of German-speaking Europe. The modern office of chancellor was established with the
Answer: Olaf Scholz"""

answer = wiki_tool_chain(model,
                         parameters,
                         context,
                         exemplar,
                         "Quien publicó el album Struggle for pleasure?",
                         show_activity = False)
print(answer)


With `show_activity = True` to see the breakdown of the LLM calls:

In [ ]:
wiki_tool_chain(model,
                parameters,
                context,
                exemplar,
                "Quien publicó el album Struggle for pleasure?",
                show_activity = True)